In [132]:
import pandas as pd

In [133]:
df = pd.read_csv('../scraped_olx_data.csv')

In [134]:
df.head()

,Title,Location,Price,Link,Image,Info
0,|| kp properties || 3bhk for rent in ramesh na...,"Ramesh Nagar, Delhi","₹ 40,000",https://www.olx.in/item/for-rent-houses-apartm...,https://apollo.olx.in:443/v1/files/o9lsyd9kipr...,3 BHK - 3 Bathroom - 1000 sqft
1,|| kp properties || 3bhk for rent in bali naga...,"Bali Nagar, Delhi","₹ 75,000",https://www.olx.in/item/for-rent-houses-apartm...,https://apollo.olx.in:443/v1/files/vciolf39les...,3 BHK - 3 Bathroom - 2000 sqft
2,2bhk for rent in ramesh nagar in prime location,"Ramesh Nagar, Delhi","₹ 33,000",https://www.olx.in/item/for-rent-houses-apartm...,https://apollo.olx.in:443/v1/files/2dh5wxpjtx4...,2 BHK - 2 Bathroom - 1000 sqft
3,1rk for rent in ramesh nagar in prime location,"Ramesh Nagar, Delhi","₹ 18,000",https://www.olx.in/item/for-rent-houses-apartm...,https://apollo.olx.in:443/v1/files/iizm9bk9nsq...,1 BHK - 1 Bathroom - 350 sqft
4,II DHRUV GOHRI II 2BHK UPP GROUND & 1ST FLOOR ...,"Ramesh Nagar, Delhi","₹ 34,000",https://www.olx.in/item/for-rent-houses-apartm...,https://apollo.olx.in:443/v1/files/i5qgfllzokw...,2 BHK - 2 Bathroom - 900 sqft


In [135]:
def clean_price(data):
    data =  data.replace('₹','').replace(',','')
    return int(data)

df['Price'] = df['Price'].apply(clean_price)

In [136]:
df.head()

,Title,Location,Price,Link,Image,Info
0,|| kp properties || 3bhk for rent in ramesh na...,"Ramesh Nagar, Delhi",40000,https://www.olx.in/item/for-rent-houses-apartm...,https://apollo.olx.in:443/v1/files/o9lsyd9kipr...,3 BHK - 3 Bathroom - 1000 sqft
1,|| kp properties || 3bhk for rent in bali naga...,"Bali Nagar, Delhi",75000,https://www.olx.in/item/for-rent-houses-apartm...,https://apollo.olx.in:443/v1/files/vciolf39les...,3 BHK - 3 Bathroom - 2000 sqft
2,2bhk for rent in ramesh nagar in prime location,"Ramesh Nagar, Delhi",33000,https://www.olx.in/item/for-rent-houses-apartm...,https://apollo.olx.in:443/v1/files/2dh5wxpjtx4...,2 BHK - 2 Bathroom - 1000 sqft
3,1rk for rent in ramesh nagar in prime location,"Ramesh Nagar, Delhi",18000,https://www.olx.in/item/for-rent-houses-apartm...,https://apollo.olx.in:443/v1/files/iizm9bk9nsq...,1 BHK - 1 Bathroom - 350 sqft
4,II DHRUV GOHRI II 2BHK UPP GROUND & 1ST FLOOR ...,"Ramesh Nagar, Delhi",34000,https://www.olx.in/item/for-rent-houses-apartm...,https://apollo.olx.in:443/v1/files/i5qgfllzokw...,2 BHK - 2 Bathroom - 900 sqft


In [137]:
df.duplicated().sum()

np.int64(6015)

In [138]:
df.drop_duplicates(inplace = True)

In [139]:
df.shape

(1945, 6)

In [140]:
df['Info'].value_counts()

Info
2 BHK - 2 Bathroom - 1000 sqft    92
1 BHK - 1 Bathroom - 450 sqft     86
2 BHK - 2 Bathroom - 650 sqft     80
2 BHK - 2 Bathroom - 900 sqft     74
1 BHK - 1 Bathroom - 500 sqft     65
                                  ..
2 BHK - 3 Bathroom - 850 sqft      1
3 BHK - 3 Bathroom - 1050 sqft     1
2 BHK - 2 Bathroom - 506 sqft      1
2 BHK - 2 Bathroom - 757 sqft      1
1 BHK - 1 Bathroom - 70 sqft       1
Name: count, Length: 409, dtype: int64

In [141]:
import re

def parse_listing(info):
    if pd.isna(info):
        return pd.Series([None, None, None])

    bhk = None
    bath = None
    area = None

    parts = info.split(' - ')
    for part in parts:
        part = part.strip().lower()

        # Extract BHK
        if 'bhk' in part:
            match = re.search(r'(\d+)\s*bhk', part)
            if match:
                bhk = int(match.group(1))

        # Extract Bathroom
        elif 'bathroom' in part:
            match = re.search(r'(\d+)\s*bathroom', part)
            if match:
                bath = int(match.group(1))

        # Extract Area
        elif 'sqft' in part or 'ft' in part:
            match = re.search(r'(\d+(\.\d+)?)\s*(sqft|ft)', part)
            if match:
                raw_area = float(match.group(1))
                unit = match.group(3)
                area = raw_area if unit == 'sqft' else raw_area * raw_area  # Convert ft² to m² if needed
                area = round(area, 2)

    return pd.Series([bhk, bath, area])

df[['BHK', 'Bathroom', 'Area']] = df['Info'].apply(parse_listing)


In [142]:
df.head()

,Title,Location,Price,Link,Image,Info,BHK,Bathroom,Area
0,|| kp properties || 3bhk for rent in ramesh na...,"Ramesh Nagar, Delhi",40000,https://www.olx.in/item/for-rent-houses-apartm...,https://apollo.olx.in:443/v1/files/o9lsyd9kipr...,3 BHK - 3 Bathroom - 1000 sqft,3.0,3.0,1000.0
1,|| kp properties || 3bhk for rent in bali naga...,"Bali Nagar, Delhi",75000,https://www.olx.in/item/for-rent-houses-apartm...,https://apollo.olx.in:443/v1/files/vciolf39les...,3 BHK - 3 Bathroom - 2000 sqft,3.0,3.0,2000.0
2,2bhk for rent in ramesh nagar in prime location,"Ramesh Nagar, Delhi",33000,https://www.olx.in/item/for-rent-houses-apartm...,https://apollo.olx.in:443/v1/files/2dh5wxpjtx4...,2 BHK - 2 Bathroom - 1000 sqft,2.0,2.0,1000.0
3,1rk for rent in ramesh nagar in prime location,"Ramesh Nagar, Delhi",18000,https://www.olx.in/item/for-rent-houses-apartm...,https://apollo.olx.in:443/v1/files/iizm9bk9nsq...,1 BHK - 1 Bathroom - 350 sqft,1.0,1.0,350.0
4,II DHRUV GOHRI II 2BHK UPP GROUND & 1ST FLOOR ...,"Ramesh Nagar, Delhi",34000,https://www.olx.in/item/for-rent-houses-apartm...,https://apollo.olx.in:443/v1/files/i5qgfllzokw...,2 BHK - 2 Bathroom - 900 sqft,2.0,2.0,900.0


In [143]:
def get_area(data):
    data = data.split(',')
    return data[0]

df['Locality'] = df['Location'].apply(get_area)

In [144]:
df.head()

,Title,Location,Price,Link,Image,Info,BHK,Bathroom,Area,Locality
0,|| kp properties || 3bhk for rent in ramesh na...,"Ramesh Nagar, Delhi",40000,https://www.olx.in/item/for-rent-houses-apartm...,https://apollo.olx.in:443/v1/files/o9lsyd9kipr...,3 BHK - 3 Bathroom - 1000 sqft,3.0,3.0,1000.0,Ramesh Nagar
1,|| kp properties || 3bhk for rent in bali naga...,"Bali Nagar, Delhi",75000,https://www.olx.in/item/for-rent-houses-apartm...,https://apollo.olx.in:443/v1/files/vciolf39les...,3 BHK - 3 Bathroom - 2000 sqft,3.0,3.0,2000.0,Bali Nagar
2,2bhk for rent in ramesh nagar in prime location,"Ramesh Nagar, Delhi",33000,https://www.olx.in/item/for-rent-houses-apartm...,https://apollo.olx.in:443/v1/files/2dh5wxpjtx4...,2 BHK - 2 Bathroom - 1000 sqft,2.0,2.0,1000.0,Ramesh Nagar
3,1rk for rent in ramesh nagar in prime location,"Ramesh Nagar, Delhi",18000,https://www.olx.in/item/for-rent-houses-apartm...,https://apollo.olx.in:443/v1/files/iizm9bk9nsq...,1 BHK - 1 Bathroom - 350 sqft,1.0,1.0,350.0,Ramesh Nagar
4,II DHRUV GOHRI II 2BHK UPP GROUND & 1ST FLOOR ...,"Ramesh Nagar, Delhi",34000,https://www.olx.in/item/for-rent-houses-apartm...,https://apollo.olx.in:443/v1/files/i5qgfllzokw...,2 BHK - 2 Bathroom - 900 sqft,2.0,2.0,900.0,Ramesh Nagar


In [ ]:
df.to_csv('../data/cleaned_data.csv',mode = 'a',header=False,index = False)

In [157]:
df = pd.read_csv('../data/cleaned_data.csv',index_col=0)

In [158]:
df.duplicated().sum()

np.int64(0)

In [159]:
df.drop_duplicates(inplace = True)

In [160]:
df.shape

(1945, 10)

In [161]:
df['Price_per_sqft'] = df['Price'] / df['Area']

In [162]:
df.head()

,Title,Location,Price,Link,Image,Info,BHK,Bathroom,Area,Locality,Price_per_sqft
0,|| kp properties || 3bhk for rent in ramesh na...,"Ramesh Nagar, Delhi",40000,https://www.olx.in/item/for-rent-houses-apartm...,https://apollo.olx.in:443/v1/files/o9lsyd9kipr...,3 BHK - 3 Bathroom - 1000 sqft,3.0,3.0,1000.0,Ramesh Nagar,40.000000
1,|| kp properties || 3bhk for rent in bali naga...,"Bali Nagar, Delhi",75000,https://www.olx.in/item/for-rent-houses-apartm...,https://apollo.olx.in:443/v1/files/vciolf39les...,3 BHK - 3 Bathroom - 2000 sqft,3.0,3.0,2000.0,Bali Nagar,37.500000
2,2bhk for rent in ramesh nagar in prime location,"Ramesh Nagar, Delhi",33000,https://www.olx.in/item/for-rent-houses-apartm...,https://apollo.olx.in:443/v1/files/2dh5wxpjtx4...,2 BHK - 2 Bathroom - 1000 sqft,2.0,2.0,1000.0,Ramesh Nagar,33.000000
3,1rk for rent in ramesh nagar in prime location,"Ramesh Nagar, Delhi",18000,https://www.olx.in/item/for-rent-houses-apartm...,https://apollo.olx.in:443/v1/files/iizm9bk9nsq...,1 BHK - 1 Bathroom - 350 sqft,1.0,1.0,350.0,Ramesh Nagar,51.428571
4,II DHRUV GOHRI II 2BHK UPP GROUND & 1ST FLOOR ...,"Ramesh Nagar, Delhi",34000,https://www.olx.in/item/for-rent-houses-apartm...,https://apollo.olx.in:443/v1/files/i5qgfllzokw...,2 BHK - 2 Bathroom - 900 sqft,2.0,2.0,900.0,Ramesh Nagar,37.777778


In [163]:
df.to_csv('../data/cleaned_data.csv')